# Testing different antenna frequencies

In [5]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
tf.get_logger().setLevel('ERROR')

# Set random seed for reproducibility
sionna.config.seed = 42

from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera, watt_to_dbm
from sionna.mimo.precoding import normalize_precoding_power, grid_of_beams_dft
from sionna.utils import log10

# Python importings
import gc
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
try:
    import pandas as pd
except:
    os.system("pip install pandas")
    import pandas as pd

## Utility functions

In [5]:
def extract_data(df, name):

    """
    This function retrieves coordinates, height, gains and tilt
    from the desired BS, given its ID
    """

    # print(df.head()) # Display the first few rows
    # print(df.dtypes)

    # preprocessing (preguntar a Rolando)        
    # filtered_df = df[(df['X(UTMHuso30)'] >= X_min) & (df['X(UTMHuso30)'] <= X_max) & (df['Y(UTMHuso30)'] >= Y_min) & (df['Y(UTMHuso30)'] <= Y_max)]
    # filtered_df.head()
    
    df_one_PROV_EMPLA = df[df["PROV_EMPLA"] == int(name)] # filter by PROV_EMPLA given by the name of the folder
    # print(f"Test whether the DF is empty or not: {df_one_PROV_EMPLA}")  # Display the filtered rows
    X_UTMHuso30 = df_one_PROV_EMPLA["X(UTMHuso30)"].tolist()
    Y_UTMHuso30 = df_one_PROV_EMPLA["Y(UTMHuso30)"].tolist()
    # print(f"Estas son las coordenadas: X_UTMHuso30: {X_UTMHuso30}")
    # print(f"Estas son las coordenadas: Y_UTMHuso30: {Y_UTMHuso30}")

    # let us convert the coordinates to an understandable format for Sionna --> relative location coordinates
    # x_location =  X_UTMHuso30 - X_min*np.ones_like(X_UTMHuso30) # ask Rolando
    # y_location =  Y_UTMHuso30 - Y_min*np.ones_like(Y_UTMHuso30)

    orientations = df_one_PROV_EMPLA["ORIENTACION"].tolist()
    gants = df_one_PROV_EMPLA["GANT"].tolist()
    alturas = df_one_PROV_EMPLA["altura"].tolist()
    tilts = df_one_PROV_EMPLA["tilt total"].tolist()

    return orientations, gants, alturas, tilts

## Main

In [6]:
cm_metric = "rss" # choose between: ["path_gain", "rss", "sinr"]

# First, let's start by retrieving the coords of the BS by getting the name of the folder and searching by its name on the dataset
# directories = ["sionna_madrid" ,"sionna_madrid_concrete_ground"]
folder = "2800005"
path = os.path.join("sionna_madrid", folder)
saving_dir = "antenna_freq_tests"

if os.path.exists(saving_dir): print(f"The path exists")

xml_file = [os.path.join(path, file) for file in os.listdir(path) if file.endswith(".xml")]

# let us get the .xml file
print(f"XML file -> {xml_file}")

# let us render the scene
# my_cam = Camera("my_cam", position=[-250,250,150], look_at=[0,0,0])
# scene.add(my_cam)
# if render_to_file:
#     scene.render_to_file(camera="my_cam", filename=os.path.join(scene_dir,"rendered_scene.png"), resolution=[650,500])
# scene.remove("my_cam") 

# let us now specify the frequency of the scene
frequencies = [1e9, 2.1e9, 2.6e9, 3.5e9]

for frequency in frequencies:
    # try to load the scene, if it fails save the log 
    # scene = sionna.rt.load_scene(xml_file[0])
    scene = load_scene(sionna.rt.scene.etoile)
    scene_size_x = scene.size[0]
    scene_size_y = scene.size[1]
    print(f"We loaded the scene")
    print(f"This is the size of the scene:{scene.size}")
    scene.frequency = float(frequency)
    print(f"This is the frequency of the scene:{scene.frequency} ")
    
    # the scene is loaded correctly, let us now retrieve the relevant antenna parameters
    df = pd.read_csv("datos_antenas_Madrid_191124.csv", sep=";") # Replace with your file path
    orientations, gants, alturas, tilts = extract_data(df,folder)
    
    # let us compute the coverage map
    num_antennas = len(orientations)
    tx_power = 0 # If we define the Tx power as 0 dBm we are going to obtain instead of received signal the pathloss
    gain_tr38901 = 8 # double check the value in case of doubt
    
    # This loop defines each transmitter
    # tensor_list = [] 
    # for i in range(num_antennas):
    i = 0
    print(f"Sector number {i}")
    transmitter = Transmitter(name=f"tx_{i+1}",
                            position=[0,0,alturas[i]],
                            orientation = [np.deg2rad(orientations[i]), np.deg2rad(tilts[i]), 0], # ToDo adjust reference angles
                            power_dbm = tx_power + gants[i] - gain_tr38901) # We are considering here the PIRE: ToDo Substract the gain of the radiation patter we use

    scene.add(transmitter)
    scene.tx_array = PlanarArray(num_rows=1,
                                num_cols=1,
                                vertical_spacing=0.5,  # relative to wavelength
                                horizontal_spacing=0.5,  # relative to wavelength
                                pattern="tr38901",
                                polarization="V")!
    

    scene.rx_array = scene.tx_array

    # define a camera to save the coverage map
    camera_1 = Camera(name = 'testing_camera', position=[0, 0, 300], look_at=[0, 0, 0])

    max_depth = 8
    num_samples = int(10e6)
    cm = scene.coverage_map(max_depth=max_depth,           # Maximum number of ray scene interactions
                            num_samples=num_samples, # If you increase: less noise, but more memory required
                            diffraction = True,
                            cm_cell_size=(2, 2),   # Resolution of the coverage map
                            cm_center=[0, 0, 1.5],   # Center of the coverage map, 1.5 m for the Rx_height
                            cm_size=[int(scene_size_x), int(scene_size_y)],    # Total size of the coverage map
                            cm_orientation=[0, 0, 0])

    # cm_image = cm.show(metric = cm_metric, vmin = -200, vmax = -50)
    cm_image = cm.show(metric = cm_metric)

    # let us save the CM
    cm_image.savefig(os.path.join(saving_dir, f"freq_{frequency}_{orientations[i]}.pdf"))

    # # let us save the CM as a tensor in both natural units and in dBs
    # cm_tensor_natural = cm.rss
    # # print(f"\n Original tensor with no conversion: {cm_tensor_natural}\n")

    # cm_tensor_dB = watt_to_dbm(cm_tensor_natural)
    # cm_tensor_dB = np.nan_to_num(cm_tensor_dB, nan=-220, posinf = -220, neginf = -220)

    # # cm_tensor_dB = 30 + 10*np.log10(cm_tensor_natural)
    # # cm_tensor_dB = np.nan_to_num(cm_tensor_dB, nan=-220, posinf = -220, neginf = -220)
    # # print(f"This is the tensor with np.log10: {cm_tensor_np_dB}")

    # # Avoid NaN values and + || - infty
    # min_val = tf.reduce_min(tf.where(tf.math.is_nan(cm_tensor_dB), tf.constant(float('inf')), cm_tensor_dB))
    # print(f"Min value of the tensor: {min_val}")
    # print(type(cm_tensor_dB))
    # print(tf.shape(cm_tensor_dB))

    # tensor_list.append(cm_tensor_dB)
    # print(f"This is the shape of the tensor with no mods:{tf.shape(cm_tensor_natural)}")
    # print(f"Content of the tensor_list: {tensor_list}")
    
    # # has_nonzero = tf.reduce_any(cm_tensor_natural != 0)
    # # print(f"Elements inside the tensor != 0:{has_nonzero} ")
    
    plt.close("all") # close the image to avoid memory consumption

    scene.remove(f"tx_{i+1}")

    # Free memory properly
    del cm, scene
    gc.collect()  # Force memory cleanup
    
    # final_tensor = tf.stack(tensor_list, axis = 0)
    # print(f"Shape of the final tensor: {tf.shape(final_tensor)}")
    # print(f"Content of the final tensor: {final_tensor}")
    
    # # let's turn it into a npy variable
    # # final_tensor_npy = final_tensor.numpy()
    
    # # let's save the tensor in a .npy file
    # np.save(os.path.join(scene_dir,"cm_tensor_dB.npy"), final_tensor.numpy().astype(np.float32))
    
    # # clean all tensor variables
    # tensor_list = [] 
    # final_tensor = []
    # tf.keras.backend.clear_session()


The path exists
XML file -> ['sionna_madrid/2800005/scene.xml']
We loaded the scene
This is the size of the scene:[853.66284 676.12054  50.     ]
This is the frequency of the scene:1000000000.0 
Sector number 0
We loaded the scene
This is the size of the scene:[853.66284 676.12054  50.     ]
This is the frequency of the scene:2100000000.0 
Sector number 0
We loaded the scene
This is the size of the scene:[853.66284 676.12054  50.     ]
This is the frequency of the scene:2600000000.0 
Sector number 0
We loaded the scene
This is the size of the scene:[853.66284 676.12054  50.     ]
This is the frequency of the scene:3500000000.0 
Sector number 0


## Let's test it now with Sionna's code

In [8]:
# Load integrated scene
frequencies = [700e6, 800e6, 2.1e9, 2.6e9, 3.5e9, 30e9]
cm_metric = 'rss'

for frequency in frequencies:
    scene = load_scene(sionna.rt.scene.munich) # Try also sionna.rt.scene.etoile

    # let us set the required bandwidth
    scene.bandwidth = 100e6
    
    # Configure antenna array for all transmitters
    scene.tx_array = PlanarArray(num_rows=1,
                                 num_cols=1,
                                 vertical_spacing=0.5,
                                 horizontal_spacing=0.5,
                                 pattern="tr38901",
                                 polarization="V")
    
    # Configure antenna array for all receivers
    scene.rx_array = PlanarArray(num_rows=1,
                                 num_cols=1,
                                 vertical_spacing=0.5,
                                 horizontal_spacing=0.5,
                                 pattern="dipole",
                                 polarization="cross")
    
    # Create transmitter
    tx = Transmitter(name="tx",
                     position=[8.5,21,27])
    
    # Add transmitter instance to scene
    scene.add(tx)
    
    # Create a receiver
    rx = Receiver(name="rx",
                  position=[45,90,1.5],
                  orientation=[0,0,0])
    
    # Add receiver instance to scene
    scene.add(rx)
    
    tx.look_at(rx) # Transmitter points towards receiver
    
    scene.frequency = frequency # in Hz; implicitly updates RadioMaterials
    print(f"This is the frequency of the scene: {frequency}")
    
    scene.synthetic_array = True # If set to False, ray tracing will be done per antenna element (slower for large arrays)
    
    cm = scene.coverage_map(max_depth=5,
                            diffraction=True, # Disable to see the effects of diffraction
                            cm_cell_size=(5., 5.), # Grid size of coverage map cells in m
                            combining_vec=None,
                            precoding_vec=None,
                            num_samples=int(10e6)) # Reduce if your hardware does not have enough memory
    
    cm_image = cm.show(metric = cm_metric); # If multiple transmitters exist, tx selects for which transmitter the cm is shown
    cm_image.savefig(f"freq_{cm_metric}_{frequency}.pdf")
    del scene, cm

This is the frequency of the scene: 700000000.0


ValueError: Material 'itu_marble' is used by the object  'Heilig_Geist-itu_marble' but is not well-defined.